In [1]:
print("ok")

ok


In [48]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
import pinecone
import os
from langchain.document_loaders import PyMuPDFLoader,PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.autonotebook import tqdm

In [51]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key=os.getenv("OPENAI_API_KEY")

In [52]:
os.environ["OPENAI_API_KEY"]=api_key


In [46]:
!pip install pinecone-client[grpc]

  Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached grpcio-1.64.1-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached lz4-4.3.3-cp311-cp311-win_amd64.whl.metadata (3.8 kB)
  Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl.metadata (1.5 kB)
Using cached googleapis_common_protos-1.63.2-py2.py3-none-any.whl (220 kB)
Using cached grpcio-1.64.1-cp311-cp311-win_amd64.whl (4.1 MB)
Using cached lz4-4.3.3-cp311-cp311-win_amd64.whl (99 kB)
Using cached protobuf-4.25.3-cp310-abi3-win_amd64.whl (413 kB)
Using cached protoc_gen_openapiv2-0.0.1-py3-none-any.whl (7.9 kB)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
## Connect to Pinecone server
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key='4f1612af-8adc-45aa-ae26-89fab99d7bb7')

index_name = "medical-chatbot"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws', 
            region='us-east-1'
        ) 
    ) 

In [50]:
def load_pdf(directory):
    loader=PyPDFDirectoryLoader(directory,glob='*.pdf')
    data=loader.load()
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
    documents=text_splitter.split_documents(data)
    return documents